# 1. 日期和时间数据类型
- datetime模块中的datetime、timedelta方法，表示为datetime.datetime，可简写为datetime；
- datetime以毫秒形式存储日期和时间，datetime(year, month, day, hour, minute), timedelta(day, second, mircosecond)

## 字符串与datetime的相互转换
- str(time)：将datetime对象和pd的Timestamp对象转换为字符串
- time.strftime('%Y-%m-%d')：将datetime对象和pd的Timestamp对象转换为字符串
    - %Y: 4位数的年
    - %y: 2位数的年
    - %m: 2位数的月
    - %d: 2位数的日
    - %H: 24小时制的时
    - %I: 12小时制的时
    - %M: 2位数的分
    - %S: 2位数的秒
    - %w: 星期几，0为星期日
    - %U: 每年的第几周[00, 53]
    - %z: 以+HHMM或-HHMM表示的UTC时区偏移量
    - %F: %Y-%m-%d简写形式
    - %D: %m/%d/%y简写形式
- datetime.strptime(time, '格式')：将字符串转换为日期，已知格式
    - 只能传入单一字符串
    - 对于字符串列表，使用[... for ...]
- dateutil.parser中的parse(time)：将字符串转换为日期，适用于常见日期格式，无需编写格式，能自动识别
    - 传入dayfirst=True，对应先‘日’后‘月’的日期格式
    - 缺点：会把一些原本不是日期的字符串认作是日期
- pd.to_datetime(time)：将字符串转换为日期
    - 可传入字符串列表
    - 可处理缺失值NaT(Not a Time)

# 2. 时间序列
- 意义：
    - 时间戳（timestamp）：特定的时刻
        - pd.Timestamp(...)/pd.date_range(...)
        - DatetimeIndex
    - 固定时期（period）：如2010年全年
        - pd.Period(...)/pd.period_range(...)
        - PeriodIndex
    - 时间间隔（interval）：由起始和结束时间戳表示，period可看做interval的特例
    - 实验或过程时间，每个时间点都是相对于特定起始时间的一个度量
- 不同索引的时间序列之间的算术运算会自动按日期对齐
- 索引、切片：
    - ts[ts.index[2]]
    - ts['日期']：可传入任何可被解释的日期格式，可为年、月或特定时刻
    - ts[datetime(...)]
    - ts.truncat(after='日期')：截掉后面的部分

# 3. 日期的范围、频率与移动
- pd.date_range('date1', 'date2', freq=)或pd.date_range(start='date1', periods=, freq=)或pd.date_range(end='date2', periods=, freq=)
- 频率freq=
    - 'D': 每日历日（默认设置）
    - 'B': 每工作日
    - 'H': 每小时
    - 'T/min': 每分
    - 'S': 每秒
    - 'L/ms': 每毫秒
    - 'U': 每微秒
    - 'M': 每月最后一个日历日
    - 'BM': 每月最后一个工作日
    - 'MS': 每月第一个日历日
    - 'BMS': 每月第一个工作日
    - 'W-MON/W-TUE/W-WED/W-THU/W-FRI/W-SAT/W-SUN': 每周几
    - 'WOM-1MON/WOM-2MON/...': 每月第一/二/三/四周的星期几
    - 'Q-JAN/Q-FEB/...': 每季度最后一个月的最后一个日历日
    - 'BQ-JAN/BQ-FEB/...': 每季度最后一个月的最后一个工作日
    - 'QS-JAN/QS-FEB/...': 每季度最后一个月的第一个日历日
    - 'BQS-JAN/BQS-FEB/...': 每季度最后一个月的第一个工作日
    - 'A-JAN/A-FEB/...': 每年指定月份的最后一个日历日
    - 'BA-JAN/BA-FEB/...': 每年指定月份的最后一个工作日
    - 'AS-JAN/AS-FEB/...': 每年指定月份的第一个日历日
    - 'BAS-JAN/BAS-FEB/...': 每年指定月份的第一个工作日
- 若'date'中带有时间信息，会默认保留；传入normalize=True，则去除时间信息
- 偏移量：与基础频率对应的对象
    - from pandas.tseries.offsets import Hour, Minute, Day, MonthEnd,...
    - offset = MonthEnd()
    - 锚点偏移量：非均匀间隔的freq，如'M'、'BM'
    - 可传入groupby作为分组键：如ts.groupby(offset.rollforward).mean()——根据移位后的日期进行分组
- 移动ts.shift()：沿着时间轴将数据迁移或后移
    - 数据的空间移位：ts.shift(2)
        - 正数向后移，负数向前移；
        - 时间索引保持不变
    - 时间的移位：
        - ts.shift(2, freq=): 根据freq修改时间序列
        - 通过偏移量移位：如date1 + 3 * Day()
            - 锚点偏移量：offset.rollforward(date)/offset.rollback(date) 前后位移

# 4. 时区处理
- pandas中的pytz库
- 获取时区对象：pytz.timezone('America/New_York')
- 时区本地化和转换
    - 时间索引中传入tz=：如pd.date_range(..., tz='UTC)
    - 本地化：ts.tz_localize('时区')
    - 转换到特定时区：ts.tz_convert('时区')
    - tz_localize和tz_convert也是DatetimeIndex的实例方法：ts.index.tz_licalize(...)
- 意识型Timestamp对象
    - stamp = pd.Timestamp(..., tz=)
    - 内部保存了一个UTC时间戳值（stamp.value），该值在时区转换过程中不会发生变化
    - 使用offset对象执行时间算术运算时，运算过程会自动关注是否存在夏令时转变期
- 不同时区之间的运算结果合为UTC

# 5. 时期及其算术运算
- pd.Period('time', freq=)/pd.period_range('time1', 'time2', freq=)
- 频率转换：p.asfreq('freq', how=)
- 季度型频率Q-JAN~Q-DEC
    - 不同的频率对应不同的财年末，如Q-SEP表示以9月为财年末月，则10-12月为Q1；
- Timestamp与Period之间的转换
    - p.to_timestamp()
    - ts.to_period()
    - 可传入新的频率，how=
- 通过数组创建PeriodIndex：pd.PeriodIndex(arr1, arr2, freq=)

# 6. 重采样
- 将时间序列从一个频率转换到另一个频率的处理过程。将高频率数据聚合到低频率称为降采样，反之称为升采样。
- ts.resample(...).mean(): 先分组，再聚合
    - freq=, 重采样的频率，可为'M'/'5min'/Second(15)
    - axis=, 默认axis=0
    - fill_method=, 升采样如何插值（'ffill'/'bfill'），默认不插值
    - closed=, 降采样中，各时间段的闭合端（right/left），默认right
    - label=, 降采样中，如何设置聚合值的标签（right/left，即面元的右/左边界），默认right
    - loffset=, 面元标签的时间校正值，比如'-1s'用于将聚合标签调早1s（移位，也可通过shift方法实现）
    - kind=, 聚合到'period'/'timestamp'
    - convention=, 当对周期进行重采样，将低频周期转换为高频的惯用法（'start'/'end'），默认'end'
- 降采样downsampling
    - closed=, label=
- OHLC重采样
    - 金融领域常用的时间序列聚合方式，即计算各面元的4个值，第一个值（open，开盘）、最后一个值（close，收盘）、最大值（high，最高）、最小值（low，最低）
    - ts.resample(...).ohlc()
- 升采样
    - 无需聚合，而是使用asfreq()方法：ts.resample(...).asfreq()
    - 需要插值：ts.resample(...).ffill()
        - 可指定填充的数量，ffill中传入limit=
        - resample中的新日期索引可以与原来的不重叠
- 对period的重采样
    - 规则更严格，在降采样中，目标频率率必须是源频率的子时期；在升采样中，目标频率必须是源频率的超时期

# 7. 移动窗口函数
- 在移动窗口上计算的各种统计函数，可以圆滑噪音数据或断裂数据；
- 会自动排除缺失值；
- data.rolling(220).mean()：表达式rolling(220)与groupby类似，但它不是分组，而是创建一个按照220天分组的滑动窗口对象
- 扩展窗口平均expanding_mean = data.expanding().mean()
    - “扩展”意味着从时间序列的起始处开始窗口，增加窗口直到它超过所有的序列
- 指数加权函数data.ewm(span=).mean()
- 自定义移动窗口函数data.rolling(...).apply(func)

In [19]:
from datetime import datetime, timedelta

now = datetime.now()
now.year, now.month, now.day

time1 = datetime(2008, 6, 24, 8, 15)
time1.minute

delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

start = datetime(2011, 1, 7)
a = timedelta(12, 5, 6)
a

start + timedelta(12)
start - 2 * timedelta(12)

datetime.timedelta(days=12, seconds=5, microseconds=6)

In [38]:
# 字符串与datetime转换

from datetime import datetime, timedelta
from dateutil.parser import parse
import pandas as pd

stamp = datetime(2011, 1, 3)
str(stamp)
stamp.strftime('%Y-%m-%d')
stamp.strftime('%F')

time = '2011-01-03'
datetime.strptime(time, '%Y-%m-%d')

time2 = ['7/6/2011', '8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in time2]

parse(time)
[parse(x) for x in time2]
parse('Jan 31, 1997 10:45 PM')

parse('13/12/2011', dayfirst=True)


time3 = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']
pd.to_datetime(time3)

idx = pd.to_datetime(time3 + [None])
idx
idx[2]

pd.isnull(idx)

array([False, False,  True])

In [64]:
from datetime import datetime
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

dates = [datetime(2011, 1, 2), datetime(2011, 1, 5), datetime(2011, 1, 7),
         datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.randn(6), index=dates)
print(ts)
ts.index

ts + ts[::2]

ts.index.dtype

stamp = ts.index[0]
stamp

ts[ts.index[2]]
ts['1/10/2011']
ts['20110110']
ts[datetime(2011, 1, 10)]
ts['1/6/2011': '20110111']
ts[datetime(2011, 1, 7):]
ts.truncate(after='1/9/2011')


long_ts = pd.Series(np.random.randn(1000),
                    index=pd.date_range('1/1/2000', periods=1000))
long_ts['2001']
long_ts['2001-05']


dates2 = pd.date_range('1/1/2020', periods=100, freq='W-WED')
df = pd.DataFrame(np.random.randn(100, 4), index=dates2,
                  columns=['Colorado', 'Texas', 'Ohio', 'LA'])
df.loc['2020-5']

2011-01-02   -0.275661
2011-01-05    0.502559
2011-01-07    0.820553
2011-01-08    0.711415
2011-01-10   -1.550497
2011-01-12   -0.360568
dtype: float64


,Colorado,Texas,Ohio,LA
2020-05-06,0.185680,0.217766,0.087393,0.891555
2020-05-13,-0.170505,-0.775085,1.321957,0.941313
2020-05-20,-0.053718,0.115333,-1.543854,-0.119955
2020-05-27,0.818905,-0.378909,-0.511516,0.219667


In [72]:
# 重复索引
from datetime import datetime
import numpy as np
import pandas as pd
from pandas import Series

dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5), index=dates)
print(dup_ts)

dup_ts.index.is_unique
dup_ts['1/3/2000']
dup_ts['1/2/2000']
dup_ts.groupby(level=0).mean()
dup_ts.groupby(level=0).count()

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32


2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

In [76]:
from datetime import datetime
import numpy as np
import pandas as pd
from pandas import Series

dates = [datetime(2011, 1, 2), datetime(2011, 1, 5), datetime(2011, 1, 7),
         datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.randn(6), index=dates)
print(ts)

re_ts = ts.resample('D')
re_ts

2011-01-02    1.551626
2011-01-05    0.621397
2011-01-07   -0.533835
2011-01-08    1.786112
2011-01-10   -0.329014
2011-01-12   -0.102187
dtype: float64


In [103]:
from datetime import datetime
import numpy as np
import pandas as pd
from pandas import Series
from pandas.tseries.offsets import Hour, Minute, Day, MonthEnd

# 范围
pd.date_range('2020-02-01', '2020-04-01')
pd.date_range(start='2020-02-01', periods=20)
pd.date_range(end='2020-05-01', periods=20)


# 频率
pd.date_range('2020-02-02', periods=10, freq='M')
pd.date_range('2020-02-02', periods=10, freq='B')


# 偏移量
pd.date_range('2020-02-02', periods=10, freq='4h')
pd.date_range('2020-02-02', periods=10, freq='1h30min')

hour = Hour()
hour
four_hours = Hour(4)
four_hours
Hour(2) + Minute(30)


# 移动
ts = pd.Series(np.random.randn(4),
               index=pd.date_range('1/1/2020', periods=4, freq='M'))
print(ts)

ts.shift(2)
ts.shift(-2)

ts.shift(2, freq='M')
ts.shift(3, freq='D')
ts.shift(1, freq='90T')


# 通过偏移量对日期进行位移
now = datetime.now()
now + 2 * Day()
now + MonthEnd()
now + MonthEnd(2)

offset = MonthEnd()
offset.rollforward(now)
offset.rollback(now)

ts2 = pd.Series(np.arange(20),
                index=pd.date_range('1/15/2020', periods=20, freq='4d'))
print(ts2)
ts2.groupby(offset.rollforward).mean()
ts2.resample('M').mean()

2020-01-31   -1.629115
2020-02-29   -0.365311
2020-03-31    1.043572
2020-04-30   -0.078215
Freq: M, dtype: float64
2020-01-15     0
2020-01-19     1
2020-01-23     2
2020-01-27     3
2020-01-31     4
2020-02-04     5
2020-02-08     6
2020-02-12     7
2020-02-16     8
2020-02-20     9
2020-02-24    10
2020-02-28    11
2020-03-03    12
2020-03-07    13
2020-03-11    14
2020-03-15    15
2020-03-19    16
2020-03-23    17
2020-03-27    18
2020-03-31    19
Freq: 4D, dtype: int32


2020-01-31     2.0
2020-02-29     8.0
2020-03-31    15.5
Freq: M, dtype: float64

In [34]:
# 时区

import numpy as np
import pandas as pd
from pandas import Series
import pytz
from pandas.tseries.offsets import Hour

pytz.timezone('America/New_York')

rng = pd.date_range('4/3/2020 9:20', periods=6, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts
ts.index.tz

#时区本地化和转换
pd.date_range('4/3/2020 9:20', periods=6, freq='D', tz='UTC')

ts_utc = ts.tz_localize('UTC')
ts_utc
ts_utc.index

ts_utc.tz_convert('America/New_York')

ts_eastern = ts.tz_localize('America/New_York')
ts_eastern.tz_convert('UTC')
ts_eastern.tz_convert('Europe/Berlin')

ts.index.tz_localize('Asia/Shanghai')

#意识型Timestamp对象
stamp = pd.Timestamp('2011-02-12 04:00')
stamp_utc = stamp.tz_localize('utc')
stamp_utc.tz_convert('America/New_York')

pd.Timestamp('2011-02-12 04:00', tz='Europe/Moscow')

stamp_utc.value
stamp_utc.tz_convert('America/New_York').value


stamp2 = pd.Timestamp('2012-11-03 01:30', tz='US/Eastern')
stamp2 + Hour()

stamp3 = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')
stamp3 + 2 * Hour()


#不同时区之间的运算
rng2 = pd.date_range('4/1/2020 9:20', periods=10, freq='B')
ts = pd.Series(np.random.randn(len(rng2)), index=rng2)
ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')
ts1 + ts2

2020-04-01 08:20:00+00:00         NaN
2020-04-02 08:20:00+00:00         NaN
2020-04-03 08:20:00+00:00   -0.594189
2020-04-06 08:20:00+00:00    0.463043
2020-04-07 08:20:00+00:00   -2.040547
2020-04-08 08:20:00+00:00    2.247075
2020-04-09 08:20:00+00:00   -1.018072
Freq: B, dtype: float64

In [ ]:
# 时期period

import numpy as np
import pandas as pd
from pandas import Series
import pytz
from pandas.tseries.offsets import Hour

p = pd.Period(2007, freq='A-DEC')
p

p + 5

pd.Period(2020, freq='A-DEC') - p

rng = pd.period_range('2020-1-1', '2020-5-5', freq='M')
rng

pd.Series(np.random.randn(5), index=rng)


values = ['2001Q3', '2010Q2', '2020Q1']
index = pd.PeriodIndex(values, freq='Q-DEC')
index


#频率转换
p.asfreq('M', how='start')
p.asfreq('M', how='end')

p2 = pd.Period('Aug-2007', 'M')
p.asfreq('A-JUN')

rng2 = pd.period_range('2006', '2010', freq='A-DEC')
ts = pd.Series(np.random.randn(len(rng2)), index=rng2)
ts.asfreq('M', how='start')
ts.asfreq('B', how='end')


#季度型频率
p3 = pd.Period('2020Q4', freq='Q-JAN')
p3.asfreq('D', 'start')
p3.asfreq('D', 'end')

p4pm = (p.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
p4pm
p4pm.to_timestamp()

rng3 = pd.period_range('2011Q3', '2012Q4', freq='Q-JAN')
ts = pd.Series(np.arange(len(rng3)), index=rng3)
ts
new_rng = (rng.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
new_rng.to_timestamp()


#Timestamp与Period之间的转换
rng4 = pd.date_range('2020-2-2', periods=3, freq='M')
ts4 = pd.Series(np.arange(3), index=rng4)
ts4.to_period()

rng5 = pd.date_range('2/2/2020', periods=6, freq='D')
ts5 = pd.Series(np.arange(6), index=rng5)
ts5.to_period('M')


#通过数组创建PeriodIndex
y = [2019, 2019, 2019, 2020, 2020]
q = [2, 3, 4, 1, 2]
index = pd.PeriodIndex(year=y, quarter=q, freq='Q-DEC')
index

In [ ]:
# 重采样

import numpy as np
import pandas as pd
from pandas import Series, DataFrame

rng = pd.date_range('2020-1-1', periods=100, freq='D')
ts = pd.Series(np.arange(len(rng)), index=rng)

ts.resample('M').mean()
ts.resample('M', kind='period').mean()


#降采样
rng2 = pd.date_range('2020-1-1', periods=12, freq='T')
ts2 = pd.Series(np.arange(12), index=rng2)
#print(ts2)
ts2.resample('5min', closed='right').sum()
ts2.resample('5T', closed='left').sum()
ts2.resample('5T', closed='right', label='right').sum()
ts2.resample('5T', closed='right', label='left').sum()

ts2.resample('5min', closed='right', label='right', loffset='-1s').sum()
ts2.resample('5T', closed='right', label='right').sum().shift(-1, freq='S')

#ohlc重采样
ts2.resample('5T').ohlc()

#升采样
frame = pd.DataFrame(np.random.randn(2, 3),
                     index=pd.date_range('1/1/2020', periods=2, freq='W-WED'),
                     columns=['Colorado', 'Ohio', 'Texas'])
frame.resample('D').ffill()
frame.resample('D').ffill(limit=2)
frame.resample('W-THU').ffill()


#时期的重采样
frame2 = pd.DataFrame(np.random.randn(24, 3),
                     index=pd.date_range('1-2020', periods=24, freq='M'),
                     columns=['Colorado', 'Ohio', 'Texas'])
annual_frame = frame2.resample('A-DEC').mean()
annual_frame.resample('Q-DEC').ffill()
annual_frame.resample('Q-DEC', convention='start').ffill()
annual_frame.resample('Q-MAR').ffill()